In [3]:
import pandas as pd 
import numpy as np 
from sklearn.linear_model import LogisticRegression

In [6]:
fg = pd.read_csv("field_goal_attempts.csv") 
players = pd.read_csv("kickers.csv")

fg = fg[(fg['season_type'] == 'Reg') & (fg['week'] <= 6)] 
fg = fg[fg['field_goal_result'].isin(['Made', 'Missed'])] 
fg['make'] = fg['field_goal_result'].map({'Made': 1, 'Missed': 0})

fg.head()

,season,season_type,week,game_date,game_key,play_id,play_sequence,player_id,field_goal_result,attempt_yards,make
232,2010,Reg,1,9/9/2010,54863,1423,53,22912,Made,41,1
233,2010,Reg,1,9/9/2010,54863,1793,67,33337,Missed,46,0
234,2010,Reg,1,9/9/2010,54863,3295,127,33337,Missed,32,0
235,2010,Reg,1,9/12/2010,54866,2073,84,21213,Made,20,1
236,2010,Reg,1,9/12/2010,54870,1564,65,27091,Missed,46,0


In [9]:
model = LogisticRegression() 
model.fit(fg[['attempt_yards']], fg['make']) 
fg['expected_make_prob'] = model.predict_proba(fg[['attempt_yards']])[:, 1] 
fg['fgoe'] = fg['make'] - fg['expected_make_prob']

In [14]:
kicker_stats = fg.groupby('player_id').agg( attempts=('make', 'count'), fgoe_total=('fgoe', 'sum'), fgoe_per_attempt=('fgoe', 'mean') ).reset_index()

kicker_stats = kicker_stats[kicker_stats['attempts'] >= 10].copy() 
kicker_stats['rating'] = kicker_stats['fgoe_total'] 
kicker_stats['rank'] = kicker_stats['rating'].rank(method='min', ascending=False).astype(int)

leaderboard = kicker_stats.merge(players[['player_id', 'player_name']], on='player_id') 
leaderboard = leaderboard[['player_id', 'player_name', 'rating', 'rank']].sort_values(by='rank') 
leaderboard.head()


,player_id,player_name,rating,rank
29,33469,STEVEN HAUSCHKA,8.089239,1
23,30403,ROBBIE GOULD,7.856362,2
13,27091,MATT BRYANT,5.911992,3
38,38701,GREG ZUERLEIN,5.398462,4
40,39470,JUSTIN TUCKER,4.403850,5


In [15]:
leaderboard.to_csv("leaderboard.csv", index=False)